# Задачи по SQL

Дана база данных StackOverflow — сервис вопросов и ответов о программировании. 
StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.
Вы будете работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах вы найдёте информацию и о более поздних оценках, которые эти посты получили. 

In [2]:
# загрузка необходимых библиотек
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine 

In [ ]:
db_config = []
with open('params_db.txt') as f:
    for line in f:
        db_config.append(line.strip())

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config[0], # имя пользователя
    db_config[1], # пароль
    db_config[2], # название сервера
    db_config[3], # порт подключения
    db_config[4], # название базы данных
)

In [ ]:
engine = create_engine(connection_string) 

In [1]:
query = '''
SELECT COUNT(id)
FROM stackoverflow.posts
WHERE post_type_id = 1 AND (score>300 OR favorites_count>=100);
'''
query_1 = pd.read_sql_query(query, con=engine)

# Нашли количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки»

In [ ]:
query = '''
SELECT ROUND(AVG(cnt))
FROM
    (SELECT COUNT(id) AS cnt,
           creation_date::date AS dt
    FROM stackoverflow.posts
    WHERE post_type_id=1 
    GROUP BY dt) cnt
WHERE dt BETWEEN '2008-11-01' AND '2008-11-18';
'''
query_1 = pd.read_sql_query(query, con=engine)

# Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно?

In [ ]:
query = '''
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users u JOIN stackoverflow.badges b ON u.id=b.user_id
WHERE CAST(u.creation_date AS date) = CAST(b.creation_date AS date)
'''
query_1 = pd.read_sql_query(query, con=engine)

# Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
query = '''
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.posts p JOIN stackoverflow.votes v ON p.id=v.post_id
JOIN stackoverflow.users u ON p.user_id=u.id

WHERE u.display_name='Joel Coehoorn' AND p.score>=1;
'''
query_1 = pd.read_sql_query(query, con=engine)

# Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
query = '''
SELECT *,
      RANK() OVER (ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id; 
'''
query_1 = pd.read_sql_query(query, con=engine)

# Выгрузите все поля таблицы vote_types. 
# Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. 
# Таблица должна быть отсортирована по полю id.

In [ ]:
query = '''
SELECT u.id,
       COUNT(v.id) AS cnt
FROM stackoverflow.users u JOIN stackoverflow.votes v ON u.id=v.user_id
JOIN stackoverflow.vote_types vt ON v.vote_type_id=vt.id
WHERE vt.name='Close'
GROUP BY  u.id

ORDER BY cnt DESC, u.id DESC

LIMIT 10
'''
query_1 = pd.read_sql_query(query, con=engine)

# Отберите 10 пользователей, которые поставили больше всего голосов типа Close. 
# Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. 
# Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
query = '''
SELECT id,
       cnt_badges,
       DENSE_RANK() OVER (ORDER BY cnt_badges DESC)
FROM
    (SELECT u.id AS id,
           COUNT(b.id) AS cnt_badges
    FROM stackoverflow.users u JOIN stackoverflow.badges b ON u.id=b.user_id
    WHERE b.creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
    GROUP BY u.id
    
    ) cnt
ORDER BY cnt_badges DESC, id
LIMIT 10;
'''
query_1 = pd.read_sql_query(query, con=engine)

# Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
# Отобразите несколько полей:
# идентификатор пользователя;
# число значков;
# место в рейтинге — чем больше значков, тем выше рейтинг.
# Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
# Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
query = '''
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id)) AS sum_sc
     FROM stackoverflow.posts
     WHERE score!=0 AND title!=''
'''
query_1 = pd.read_sql_query(query, con=engine)

# Сколько в среднем очков получает пост каждого пользователя?
# Сформируйте таблицу из следующих полей:
# заголовок поста;
# идентификатор пользователя;
# число очков поста;
# среднее число очков пользователя за пост, округлённое до целого числа.
# Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
query = '''
SELECT p.title
FROM
    (SELECT u.id AS id,
       COUNT(b.id) AS cnt_badges
    FROM stackoverflow.users u JOIN stackoverflow.badges b ON u.id=b.user_id

    GROUP BY u.id
    ORDER BY cnt_badges DESC) cnt JOIN stackoverflow.posts p ON cnt.id=p.user_id
WHERE cnt.cnt_badges>1000 AND  p.title!=''
'''
query_1 = pd.read_sql_query(query, con=engine)

# Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. 
# Посты без заголовков не должны попасть в список.

In [ ]:
query = '''
SELECT id,
       views,
      
       CASE  WHEN views >= 350 THEN 1
             WHEN views < 350 AND views >= 100 THEN 2
             WHEN views < 100 AND views > 0 THEN 3
       
      END     
FROM stackoverflow.users
WHERE location LIKE '%United States%' AND views > 0 
'''
query_1 = pd.read_sql_query(query, con=engine)

# Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). 
# Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
# пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
# пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
# пользователям с числом просмотров меньше 100 — группу 3.
# Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. 
# Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
query = '''
WITH max_group AS (SELECT a.group,
                           MAX(a.views) AS max_views
                    FROM
                           (SELECT id,
                                   views,
                                   CASE  WHEN views >= 350 THEN 1
                                         WHEN views < 350 AND views >= 100 THEN 2
                                         WHEN views < 100 AND views > 0 THEN 3
       
                                  END AS group    
                            FROM stackoverflow.users
                            WHERE location LIKE '%United States%' AND views > 0) a
                    GROUP BY a.group)
SELECT DISTINCT u.id, max_group.group, u.views
FROM max_group  INNER JOIN stackoverflow.users u ON max_group.max_views = u.views
 WHERE location LIKE '%United States%' AND views > 0
ORDER BY u.views DESC, u.id;
'''
query_1 = pd.read_sql_query(query, con=engine)

# Дополните предыдущий запрос. 
# Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. 
# Выведите поля с идентификатором пользователя, группой и количеством просмотров. 
# Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
query = '''
SELECT DISTINCT EXTRACT(DAY FROM creation_date) AS dt, 
       COUNT(id) OVER(PARTITION BY DATE_TRUNC('day',creation_date)::date) AS cnt_day,
       COUNT(id) OVER(ORDER BY DATE_TRUNC('day',creation_date)::date) AS sum_cnt_day
FROM stackoverflow.users
WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'
'''
query_1 = pd.read_sql_query(query, con=engine)

# Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года.
# Сформируйте таблицу с полями:
# номер дня;
# число пользователей, зарегистрированных в этот день;
# сумму пользователей с накоплением.

In [ ]:
query = '''
SELECT DISTINCT u.id,
      
       ( FIRST_VALUE(p.creation_date) OVER (PARTITION BY p.user_id ORDER BY p.creation_date)-u.creation_date) AS div
FROM stackoverflow.posts p JOIN stackoverflow.users u ON p.user_id=u.id       
'''
query_1 = pd.read_sql_query(query, con=engine)

# Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста.
# Отобразите:
# идентификатор пользователя;
# разницу во времени между регистрацией и первым постом.

In [ ]:
query = '''
SELECT DATE_TRUNC('month',creation_date)::date AS month,
       SUM(views_count) AS sum_m
FROM stackoverflow.posts       
WHERE creation_date::date BETWEEN '2008-01-01' AND '2008-12-31'       
GROUP BY month
ORDER BY sum_m DESC  
'''
query_1 = pd.read_sql_query(query, con=engine)

# Выведите общую сумму просмотров постов за каждый месяц 2008 года. 
# Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. 
# Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
query = '''
SELECT u.display_name,
       COUNT(DISTINCT user_id)  
FROM stackoverflow.posts p
JOIN stackoverflow.users u ON u.id=p.user_id
WHERE parent_id != 0 AND p.creation_date::date BETWEEN ((u.creation_date::date)) 
                                               AND ((u.creation_date::date) + INTERVAL '1 month')
GROUP BY u.display_name
HAVING COUNT(p.parent_id) > 100
ORDER BY u.display_name ASC  
'''
query_1 = pd.read_sql_query(query, con=engine)

# Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов.
# Вопросы, которые задавали пользователи, не учитывайте. 
# Для каждого имени пользователя выведите количество уникальных значений user_id. 
# Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
query = '''
SELECT CAST(DATE_TRUNC('month', p.creation_date) AS date) AS month_num,
       COUNT(p.id) AS posts_cnt
FROM stackoverflow.posts AS p JOIN stackoverflow.users AS u ON u.id=p.user_id

WHERE p.user_id IN 
        (SELECT u.id AS user_id
          FROM stackoverflow.posts p
                LEFT JOIN stackoverflow.users u ON u.id=p.user_id
                WHERE u.creation_date::date BETWEEN '2008-09-01' AND '2008-09-30'
                       AND p.creation_date::date BETWEEN '2008-12-01' AND '2008-12-31')
GROUP BY month_num
ORDER BY month_num DESC  
'''
query_1 = pd.read_sql_query(query, con=engine)

# Выведите количество постов за 2008 год по месяцам. 
# Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года.
# Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
query = '''
SELECT user_id,
       creation_date,
       views_count,
      SUM(views_count) OVER(PARTITION BY user_id ORDER BY creation_date) AS cnt_views
FROM stackoverflow.posts       
ORDER BY user_id, creation_date   
'''
query_1 = pd.read_sql_query(query, con=engine)

# Используя данные о постах, выведите несколько полей:
# идентификатор пользователя, который написал пост;
# дата создания поста;
# количество просмотров у текущего поста;
# сумму просмотров постов автора с накоплением.
# Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей,
# а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
query = '''
SELECT ROUND(AVG(avg_activity))
FROM
       (WITH t1 AS 
           (SELECT CAST(DATE_TRUNC('day', creation_date) AS date) AS days,
                   user_id,
                   COUNT(id) AS activity_count
            FROM stackoverflow.posts
            WHERE creation_date BETWEEN '2008-12-01' AND '2008-12-07'
            GROUP BY days, user_id)
        SELECT AVG(activity_count) OVER(PARTITION BY user_id ORDER BY days) AS avg_activity
        FROM t1) AS t2  
'''
query_1 = pd.read_sql_query(query, con=engine)

# Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? 
# Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. 
# Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
query = '''
SELECT EXTRACT(MONTH FROM month_num) AS month_num,
       CAST(posts_cnt AS numeric) ,
       
       ROUND(100*(CAST(posts_cnt AS numeric)-CAST(prew AS numeric))/CAST(prew AS numeric),2)
FROM
    (SELECT CAST(DATE_TRUNC('month', p.creation_date) AS date) AS month_num,
           COUNT(p.id) AS posts_cnt,
           LAG(COUNT(p.id)) OVER (ORDER BY CAST(DATE_TRUNC('month', p.creation_date) AS date)) AS prew
      FROM stackoverflow.posts p
      WHERE p.creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'                    
      GROUP BY month_num
      ) a  
'''
query_1 = pd.read_sql_query(query, con=engine)

# На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? 
# Отобразите таблицу со следующими полями:
# номер месяца;
# количество постов за месяц;
# процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.
# Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. 
# Округлите значение процента до двух знаков после запятой.

In [ ]:
query = '''
SELECT DISTINCT(EXTRACT(WEEK FROM creation_date)),
       LAST_VALUE(creation_date) OVER(ORDER BY (EXTRACT(WEEK FROM creation_date)))
FROM stackoverflow.posts       
WHERE user_id IN (SELECT user_id
                 FROM stackoverflow.posts
                 GROUP BY user_id
                 ORDER BY COUNT(id) DESC
                 LIMIT 1) 
AND creation_date BETWEEN '2008-10-01 00:00:00' AND '2008-11-01 00:00:00';        
'''
query_1 = pd.read_sql_query(query, con=engine)

# Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. 
# Выведите данные за октябрь 2008 года в таком виде:
# номер недели;
# дата и время последнего поста, опубликованного на этой неделе.